In [25]:
import sagemaker

# Get the execution role used by the current SageMaker notebook
role = sagemaker.get_execution_role()

# Print the role
print(f"SageMaker execution role: {role}")

SageMaker execution role: arn:aws:iam::600627350106:role/service-role/AmazonSageMakerServiceCatalogProductsUseRole


In [26]:
import pandas as pd
pd.read_csv("s3://kg-demo-db/edges.csv").head()

,~id,~from,~to,~label,since
0,10,1,2,knows,2020
1,11,2,3,knows,2019
2,12,3,4,visited,2018
3,13,1,5,visited,2021


In [27]:
%load --source s3://kg-demo-db/edges.csv -p OVERSUBSCRIBE --store-to load

Button(description='Submit', style=ButtonStyle())

Output()

In [4]:
pd.read_csv("s3://kg-demo-db/vertices.csv").head()

,~id,~label,name,age
0,1,person,Alice,30.0
1,2,person,Bob,25.0
2,3,person,Charlie,35.0
3,4,city,London,NaN
4,5,city,Paris,NaN


In [5]:
%load --source s3://kg-demo-db/vertices.csv -p OVERSUBSCRIBE --store-to load

Button(description='Submit', style=ButtonStyle())

Output()

In [10]:
%%gremlin
g.V().hasLabel('person').limit(2).valueMap(true)

In [9]:
%%gremlin
g.E().hasLabel('knows').limit(2).valueMap(true)

In [15]:
from gremlin_python.driver import client
from gremlin_python.driver.protocol import GremlinServerError
from langchain_core.prompts.prompt import PromptTemplate
from langchain.chains.llm import LLMChain

import boto3
from langchain_aws import ChatBedrock
from pydantic import BaseModel, Field
from typing import List

# Set up Amazon Bedrock Boto3 client
bedrock_runtime = boto3.client(service_name="bedrock-runtime", region_name="us-east-1")

# Configure the model to use
model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
model_kwargs = {
    "max_tokens": 2048,
    "temperature": 0.1,
    "top_k": 250,
    "top_p": 1,
    "stop_sequences": ["\n\nHuman"],
}

# Create the LangChain Chat object
llm = ChatBedrock(
    client=bedrock_runtime,
    model_id=model_id,
    model_kwargs=model_kwargs,
)

neptune_endpoint = "db-neptune-1.cluster-ro-choicos2alvz.us-east-1.neptune.amazonaws.com"
neptune_port = 8182

# Replace NeptuneGraph with a custom class
class CustomNeptuneGraph:
    def __init__(self, host, port, use_https=True):
        protocol = 'wss' if use_https else 'ws'
        self.endpoint = f"{protocol}://{host}:{port}/gremlin"
        self.client = client.Client(self.endpoint, 'g')

    def query(self, query_string):
        try:
            result = self.client.submit(query_string).all().result()
            return result
        except GremlinServerError as e:
            print(f"Error executing query: {e}")
            return None

# Set up the custom Neptune graph connection
graph = CustomNeptuneGraph(
    host=neptune_endpoint,
    port=neptune_port,
    use_https=True
)

# The rest of your code remains the same
prompt_template = """
Given the following question about an airport database:
{question}

Generate ONLY the Gremlin query to answer this question. The database has vertices labeled as 'person' 
and edges labeled as 'knows'. Do not include any explanations or comments, just the raw Gremlin query.

Gremlin Query:
"""

prompt = PromptTemplate(template=prompt_template, input_variables=["question"])

# Create an LLMChain
chain = LLMChain(llm=llm, prompt=prompt)

# Function to ask questions using the chain and execute the query
def ask_question(question):
    # Generate the Gremlin query
    response = chain.run(question)
    
    # Extract only the Gremlin query (remove any explanations or comments)
    query = response.strip().split('\n')[-1]  # Get the last line of the response
    print(f"Generated Query: {query}")
    
    # Execute the query on the Neptune database
    result = graph.query(query)
    return result


<ipython-input-15-6b64c20ecd0b>:70: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


In [23]:
# Example question
question = "Whom Alice knows?"
response = ask_question(question)
print(f"Query Result: {response}")

<ipython-input-15-6b64c20ecd0b>:75: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chain.run(question)


Generated Query: g.V().has('person','name','Alice').outE('knows').inV().values('name')
Query Result: ['Bob']
